# Messi vs Ronaldo - An Analytical Debate

1. Get data from [Understat](https://understat.com/) for Messi and Ronaldo
3. Data Cleaning and EDA
3. Comparative Analysis
3. Shot Analysis
2. Distance Analysis
3. The Final comparison
3. Who Won?

In [1]:
# Import required libraries:
import json
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

# Import plotting libraries:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get that Data!

## Cristiano Ronaldo

### Season-wise data

In [8]:
# Scrape Cristiano player stats:
cris_scrape_url = "https://understat.com/player/2371"
page_connect = urlopen(cris_scrape_url)
page_html = BeautifulSoup(page_connect, "html.parser")

json_raw_string = page_html.findAll(name="script")[1].text
start_ind = json_raw_string.index("\\")
stop_ind = json_raw_string.index("')")

json_data = json_raw_string[start_ind:stop_ind]
json_data = json_data.encode("utf8").decode("unicode_escape")

cris_season_wise_meta_df = pd.json_normalize(json.loads(json_data)["season"])
cris_season_wise_meta_df.insert(0, "Player", "Cristiano")

ValueError: substring not found

In [ ]:
cris_season_wise_meta_df

### Shots Data

In [ ]:
# Scrape Cristiano shot data:
json_raw_string = page_html.findAll(name="script")[3].text
start_ind = json_raw_string.index("\\")
stop_ind = json_raw_string.index("')")

json_data = json_raw_string[start_ind:stop_ind]
json_data = json_data.encode("utf8").decode("unicode_escape")

cris_shots_df = pd.json_normalize(json.loads(json_data))
cris_shots_df.insert(0, "Player", "Cristiano")

In [ ]:
cris_shots_df

In [ ]:
cris_shots_df.shape

## Messi

### Season-wise data

In [ ]:
# Scrape Messi player stats:
messi_scrape_url = "https://understat.com/player/2097"
page_connect = urlopen(messi_scrape_url)
page_html = BeautifulSoup(page_connect, "html.parser")

json_raw_string = page_html.findAll(name="script")[1].text
start_ind = json_raw_string.index("\\")
stop_ind = json_raw_string.index("')")

json_data = json_raw_string[start_ind:stop_ind]
json_data = json_data.encode("utf8").decode("unicode_escape")

messi_season_wise_meta_df = pd.json_normalize(json.loads(json_data)["season"])
messi_season_wise_meta_df.insert(0, "Player", "Messi")

In [ ]:
messi_season_wise_meta_df

### Shots Data

In [ ]:
# Scrape Messi shot data:
json_raw_string = page_html.findAll(name="script")[3].text
start_ind = json_raw_string.index("\\")
stop_ind = json_raw_string.index("')")

json_data = json_raw_string[start_ind:stop_ind]
json_data = json_data.encode("utf8").decode("unicode_escape")

messi_shots_df = pd.json_normalize(json.loads(json_data))
messi_shots_df.insert(0, "Player", "Messi")

In [ ]:
messi_shots_df

## Build Final Season-level and shots data

In [ ]:
cris_season_wise_meta_df.drop(index=[0])

In [ ]:
cris_season_wise_meta_df.reset_index(drop=True, inplace=True)

### Joint Season-wise Data

In [ ]:
season_wise_meta_df = cris_season_wise_meta_df.append(messi_season_wise_meta_df)
season_wise_meta_df

### Joint Shots Data

In [ ]:
cris_shots_df.tail()

In [ ]:
cris_shots_df.drop(index=[1342])

In [ ]:
cris_shots_df.drop(index=[1342], inplace=True)

In [ ]:
shots_df = cris_shots_df.append(messi_shots_df)
shots_df

# Data Cleaning and EDA

## On Season-wise data

In [6]:
season_wise_meta_df.describe(include="all")

NameError: name 'season_wise_meta_df' is not defined

In [ ]:
season_wise_meta_df.info()

In [ ]:
season_wise_meta_df["goals"].sum()

In [ ]:
season_wise_meta_df.apply(pd.to_numeric, errors="ignore")

In [ ]:
season_wise_meta_df = season_wise_meta_df.apply(pd.to_numeric, errors="ignore")

In [ ]:
season_wise_meta_df.info()

In [ ]:
season_wise_meta_df["goals"].sum()

In [ ]:
type(season_wise_meta_df["goals"].sum())

In [ ]:
season_wise_meta_df = season_wise_meta_df.round(2)

In [ ]:
season_wise_meta_df

In [ ]:
season_wise_meta_df.describe(percentiles=[]).round(1)

In [ ]:
shots_df.head()

In [ ]:
season_wise_meta_df

In [ ]:
pd.set_option("display.max_rows", 120)

In [ ]:
season_wise_meta_df.groupby(["Player"]).describe(percentiles=[]).T.round(1)

## On Shots data

In [ ]:
shots_df.info()

In [ ]:
shots_df = shots_df.apply(pd.to_numeric, errors="ignore")

In [ ]:
shots_df = shots_df.round(2)

In [ ]:
shots_df.describe(percentiles=[]).round(1)

In [ ]:
shots_df

# Comparative Analysis

## On Season-wise Data

In [ ]:
season_wise_meta_df.select_dtypes(exclude=[object]).columns

In [ ]:
num_cols = season_wise_meta_df.select_dtypes(exclude=[object]).columns

In [ ]:
num_cols

In [ ]:
num_cols.__len__()

In [ ]:
len(num_cols)

In [ ]:
season_wise_meta_df.groupby(["Player"])[num_cols].sum()

In [ ]:
season_wise_meta_df.groupby(["Player"])[num_cols].sum().reset_index()

In [ ]:
total_df = season_wise_meta_df.groupby(["Player"])[num_cols].sum().round(1).reset_index()

In [ ]:
total_df

### CA on Totals (Consolidated data for all seasons per player)

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=3, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=num_cols)
# Add bar plot for goals conceded in all subplots:
for col in num_cols:
    fig.add_trace(go.Bar(x=total_df["Player"],
                        y=total_df[col],
                        text=total_df[col],
                        textposition="inside",
                        name=col),
                row=i, col=j)
    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_layout(height=800)
fig.show()

NOTE: On xGChain and xGBuildup \\
xGChain --> Calculating xG for possessions that lead to a shot where the player was involved at least once in that possession, **INCLUDING** the final pass or the shot. \\
xGBuildup --> Calculating xG for possessions that lead to a shot where the player was involved at least once in that possession, **EXCLUDING** the final pass or the shot. \\

### Feature Engineering

In [ ]:
season_wise_meta_df.columns

In [ ]:
season_wise_meta_df["xGDiff"] = season_wise_meta_df["goals"].sub(season_wise_meta_df["xG"]).round(2)

season_wise_meta_df["goal_contrib"] = season_wise_meta_df["goals"].add(season_wise_meta_df["assists"]).round(1)

season_wise_meta_df["xgoal_contrib"] = season_wise_meta_df["xG"].add(season_wise_meta_df["xA"]).round(1)

season_wise_meta_df["shot_contrib"] = season_wise_meta_df[["shots", "key_passes", "assists"]].sum(axis=1).round(1)

In [ ]:
num_cols = num_cols.tolist() + ["xGDiff", "goal_contrib", "xgoal_contrib", "shot_contrib"]

In [ ]:
num_cols

In [ ]:
num_cols = [col for col in num_cols if col not in ["season", "yellow", "red", "games"]]

In [ ]:
num_cols

In [ ]:
total_df = season_wise_meta_df.groupby(["Player"])[num_cols].sum().round(1).reset_index()

In [ ]:
num_cols.__len__()

### CA on Totals - Updated

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=3, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=num_cols)
# Add bar plot for goals conceded in all subplots:
for col in num_cols:
    fig.add_trace(go.Bar(x=total_df["Player"],
                        y=total_df[col],
                        text=total_df[col],
                        textposition="inside",
                        name=col),
                row=i, col=j)
    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_layout(height=800)
fig.show()

### Per 90 Stats

In [ ]:
for col in num_cols:
    total_df[col + "_p90"] = total_df[col].divide(total_df["time"]).multiply(90).round(2)

In [ ]:
per_90_cols = [col + "_p90" for col in num_cols if col != "time"]

In [ ]:
len(per_90_cols)

### CA on Totals - Updated

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=3, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=per_90_cols)
# Add bar plot for goals conceded in all subplots:
for col in per_90_cols:
    fig.add_trace(go.Bar(x=total_df["Player"],
                        y=total_df[col],
                        text=total_df[col],
                        textposition="inside",
                        name=col),
                row=i, col=j)
    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_layout(height=800)
fig.show()

### CA on Season-wise stats (Consolidated data per season per player)

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=3, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=num_cols)
# Add bar plot for goals conceded in all subplots:
for col in num_cols:
    cris_plot_data = season_wise_meta_df.loc[season_wise_meta_df["Player"] == "Cristiano"]
    fig.add_trace(go.Bar(x=cris_plot_data["season"],
                         y=cris_plot_data[col],
                         text=cris_plot_data[col],
                         textposition="inside",
                         name=col + "(Cris)"),
                row=i, col=j)
    messi_plot_data = season_wise_meta_df.loc[season_wise_meta_df["Player"] == "Messi"]
    fig.add_trace(go.Bar(x=messi_plot_data["season"],
                        y=messi_plot_data[col],
                        text=messi_plot_data[col],
                        textposition="inside",
                        name=col + "(Messi)"),
            row=i, col=j)

    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_layout(height=800, barmode="stack")
fig.show()

### Per 90 Stats (Season-wise)

In [ ]:
for col in num_cols:
    season_wise_meta_df[col + "_p90"] = season_wise_meta_df[col].divide(season_wise_meta_df["time"]).multiply(90).round(2)

In [ ]:
per_90_cols = [col + "_p90" for col in num_cols if col != "time"]

In [ ]:
len(per_90_cols)

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=3, cols=5,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=per_90_cols)
# Add bar plot for goals conceded in all subplots:
for col in per_90_cols:
    cris_plot_data = season_wise_meta_df.loc[season_wise_meta_df["Player"] == "Cristiano"]
    fig.add_trace(go.Bar(x=cris_plot_data["season"],
                         y=cris_plot_data[col],
                         text=cris_plot_data[col],
                         textposition="inside",
                         name=col + "(Cris)"),
                row=i, col=j)
    messi_plot_data = season_wise_meta_df.loc[season_wise_meta_df["Player"] == "Messi"]
    fig.add_trace(go.Bar(x=messi_plot_data["season"],
                        y=messi_plot_data[col],
                        text=messi_plot_data[col],
                        textposition="inside",
                        name=col + "(Messi)"),
            row=i, col=j)

    j += 1
    if j > 5:
        j = 1
        i += 1
fig.update_layout(height=800, barmode="stack")
fig.show()

## On Shots Data

#### CA on Totals

In [ ]:
shots_df.head()

In [ ]:
shots_df["result"].unique()

In [ ]:
shots_df.columns

In [ ]:
shot_cols = ["result", "situation", "shotType"]

In [ ]:
plot_df = shots_df.groupby(["Player", "result"]).agg({"result": "count"})

In [ ]:
plot_df.name = "total"

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=1, cols=3,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=shot_cols)
# Add bar plot for goals conceded in all subplots:
for col in shot_cols:
    fig.add_trace(go.Histogram(x=cris_shots_df[col],
                               text=cris_shots_df[col],
                               name=col + "(Cris)"),
                  row=i, col=j)
    fig.add_trace(go.Histogram(x=messi_shots_df[col],
                            text=messi_shots_df[col],
                            name=col + "(Messi)"),
                row=i, col=j)

    j += 1
    if j > 3:
        j = 1
        i += 1
fig.update_layout(height=400, barmode="group")
fig.show()

### CA on Per90

In [ ]:
cris_time = total_df.loc[total_df["Player"] == "Cristiano", "time"].item()
messi_time = total_df.loc[total_df["Player"] == "Messi", "time"].item()

In [ ]:
cris_time

In [ ]:
cris_shots_df["result"].value_counts().reset_index()

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=1, cols=3,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=shot_cols)
# Add bar plot for goals conceded in all subplots:
for col in shot_cols:
    cris_plot_df = cris_shots_df[col].value_counts().reset_index()
    cris_plot_df[col + "_p90"] = cris_plot_df[col].divide(cris_time).multiply(90).round(1)
    fig.add_trace(go.Bar(x=cris_plot_df["index"],
                         y=cris_plot_df[col + "_p90"],
                         text=cris_plot_df[col + "_p90"],
                         textposition="inside",
                         name=col),
                  row=i, col=j)

    messi_plot_df = messi_shots_df[col].value_counts().reset_index()
    messi_plot_df[col + "_p90"] = messi_plot_df[col].divide(messi_time).multiply(90).round(1)
    fig.add_trace(go.Bar(x=messi_plot_df["index"],
                         y=messi_plot_df[col + "_p90"],
                         text=messi_plot_df[col + "_p90"],
                         textposition="inside",
                         name=col),
                  row=i, col=j)

    j += 1
    if j > 3:
        j = 1
        i += 1
fig.update_layout(height=400, barmode="group")
fig.show()

### CA on Percentages

In [ ]:
i = j = 1
# Create an empty subplot:
fig = make_subplots(rows=1, cols=3,
                    shared_xaxes=False,
                    vertical_spacing=0.1,
                    subplot_titles=shot_cols)
# Add bar plot for goals conceded in all subplots:
for col in shot_cols:
    cris_plot_df = cris_shots_df[col].value_counts(normalize=True).multiply(100).round(1).reset_index()
    fig.add_trace(go.Bar(x=cris_plot_df["index"],
                         y=cris_plot_df[col],
                         text=cris_plot_df[col].apply(lambda x: '{0:1.2f}%'.format(x)),
                         textposition="inside",
                         name=col + "(Cris)"),
                  row=i, col=j)

    messi_plot_df = messi_shots_df[col].value_counts(normalize=True).multiply(100).round(1).reset_index()
    fig.add_trace(go.Bar(x=messi_plot_df["index"],
                         y=messi_plot_df[col],
                         text=messi_plot_df[col].apply(lambda x: '{0:1.2f}%'.format(x)),
                         textposition="inside",
                         name=col+ "(Messi)"),
                  row=i, col=j)

    j += 1
    if j > 3:
        j = 1
        i += 1
fig.update_layout(height=400, barmode="group",
                  plot_bgcolor="#383838",
                  template="plotly_dark")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
cris_shots_df.shape

# Trivia Time! 

In [5]:
fig = px.histogram(data_frame=shots_df, x="player_assisted", color="Player",
             barmode="group", title="Assisting Player Comparison",
             labels={"result": "", "count": ""})
fig.update_layout(xaxis={"tickmode": "linear"})

NameError: name 'shots_df' is not defined

# Shot Analysis

## Data Preparation

In [ ]:
!pip install mplsoccer

In [ ]:
import mplsoccer

In [ ]:
pitch = mplsoccer.Pitch(pitch_type="wyscout")
fig, ax = pitch.draw(figsize=(14, 6))

In [ ]:
shots_df["X"] = shots_df["X"].multiply(100)
shots_df["Y"] = shots_df["Y"].multiply(100)

In [ ]:
crisPlotData = shots_df[shots_df["Player"] == "Cristiano"].copy(deep=True)

In [ ]:
messiPlotData = shots_df[shots_df["Player"] == "Messi"].copy(deep=True)

## Plot HeatMaps

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
pitch = mplsoccer.VerticalPitch(pitch_type="wyscout",
                                line_color='white',
                                line_zorder=2,
                                pitch_color='#101010',
                                half=True)
fig, ax = pitch.draw(figsize=(8, 8))
bin_statistic = pitch.bin_statistic(crisPlotData["X"],
                                    crisPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='hot', edgecolors='#22312b')
ax.set_title("Cristiano Shot HeatMap", fontsize=30)

In [ ]:
pitch = mplsoccer.VerticalPitch(pitch_type="wyscout",
                                line_color='white',
                                line_zorder=2,
                                pitch_color='#101010',
                                half=True)
fig, ax = pitch.draw(figsize=(8, 8))
bin_statistic = pitch.bin_statistic(messiPlotData["X"],
                                    messiPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='hot', edgecolors='#22312b')
ax.set_title("Messi Shot HeatMap", fontsize=30)

In [ ]:
pitch = mplsoccer.VerticalPitch(pitch_type="wyscout",
                                line_color='white',
                                line_zorder=2,
                                pitch_color='#101010',
                                half=True)
fig, axs = pitch.grid(nrows=1, ncols=2,
                     endnote_height=0.03, endnote_space=0,
                      grid_width=0.88, left=0.025,
                      title_height=0.06, title_space=0,
                      axis=False,
                      grid_height=0.86)

# Cristiano:
ax = axs["pitch"][0]
bin_statistic = pitch.bin_statistic(crisPlotData["X"],
                                    crisPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Cristiano Shot HeatMap", fontsize=30)

# Messi:
ax = axs["pitch"][1]
bin_statistic = pitch.bin_statistic(messiPlotData["X"],
                                    messiPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Messi Shot HeatMap", fontsize=30)

In [ ]:
shots_df.situation.unique()

In [ ]:
""" Shot Heatmaps (excluding penalties) """
wopCrisPlotData = crisPlotData.loc[crisPlotData["situation"] != "Penalty"].copy(deep=True)
wopMessiPlotData = messiPlotData.loc[messiPlotData["situation"] != "Penalty"].copy(deep=True)

pitch = mplsoccer.VerticalPitch(pitch_type="wyscout",
                                line_color='white',
                                line_zorder=2,
                                pitch_color='#101010',
                                half=True)
fig, axs = pitch.grid(nrows=1, ncols=2,
                     endnote_height=0.03, endnote_space=0,
                      grid_width=0.88, left=0.025,
                      title_height=0.06, title_space=0,
                      axis=False,
                      grid_height=0.86)

# Cristiano:
ax = axs["pitch"][0]
bin_statistic = pitch.bin_statistic(wopCrisPlotData["X"],
                                    wopCrisPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Cristiano Shot HeatMap", fontsize=30)

# Messi:
ax = axs["pitch"][1]
bin_statistic = pitch.bin_statistic(wopMessiPlotData["X"],
                                    wopMessiPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Messi Shot HeatMap", fontsize=30)

In [ ]:
""" Shot Heatmaps (including and excluding penalties) """
pitch = mplsoccer.VerticalPitch(pitch_type="wyscout",
                                line_color='white',
                                line_zorder=2,
                                pitch_color='#101010',
                                half=True)
fig, axs = pitch.grid(nrows=2, ncols=2,
                      endnote_height=0.01, endnote_space=0,
                      grid_width=0.95, left=0.02,
                      title_height=0.04, title_space=0,
                      axis=False,
                      grid_height=0.95)

# Cristiano with pen:
ax = axs["pitch"][0, 0]
bin_statistic = pitch.bin_statistic(crisPlotData["X"],
                                    crisPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Cristiano Shot HeatMap (All Shots)", fontsize=20)

# Messi with pen:
ax = axs["pitch"][0, 1]
bin_statistic = pitch.bin_statistic(messiPlotData["X"],
                                    messiPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Messi Shot HeatMap (All Shots)", fontsize=20)

# Cristiano without pen:
ax = axs["pitch"][1, 0]
bin_statistic = pitch.bin_statistic(wopCrisPlotData["X"],
                                    wopCrisPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Cristiano Shot HeatMap (w/o Pens)", fontsize=20)

# Messi wo pen:
ax = axs["pitch"][1, 1]
bin_statistic = pitch.bin_statistic(wopMessiPlotData["X"],
                                    wopMessiPlotData["Y"],
                                    statistic='count', bins=(35, 35))
bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
pcm = pitch.heatmap(bin_statistic, ax=ax,
                    cmap='coolwarm', edgecolors='#22312b')
ax.set_title("Messi Shot HeatMap (w/o Pens)", fontsize=20)

# Distance Analysis

## Per 90 Columns

In [ ]:
season_wise_meta_df.columns

In [ ]:
num_cols

In [ ]:
for col in num_cols:
    season_wise_meta_df[col + "_p90"] = season_wise_meta_df[col].divide(season_wise_meta_df["time"]).multiply(90).round(2)

## Goals vs Shots (per season)

In [ ]:
season_wise_meta_df.head()

In [ ]:
fig = px.scatter(x="shots_p90", y="goals_p90", data_frame=season_wise_meta_df,
                 hover_name="season", text="Player", symbol="season", color="season", size="npg_p90",
                 opacity=.8)
fig.add_trace(go.Scatter(x=[season_wise_meta_df["shots_p90"].min(), season_wise_meta_df["shots_p90"].max()],
                         y=[season_wise_meta_df["goals_p90"].mean(), season_wise_meta_df["goals_p90"].mean()],
                         name="Avg. Goals"))
fig.add_trace(go.Scatter(x=[season_wise_meta_df["shots_p90"].mean(), season_wise_meta_df["shots_p90"].mean()],
                         y=[season_wise_meta_df["goals_p90"].min(), season_wise_meta_df["goals_p90"].max()],
                         name="Avg. Shots"))
fig.update_traces(textposition='top center', textfont_size=10)
fig.update_layout(showlegend=True, height=700, width=1200)
fig.update(layout_coloraxis_colorbar_len=0.5, layout_coloraxis_colorbar_y=0.4)
fig.show()

## Total Goals vs Total Expected Goals (per season)

In [ ]:
fig = px.scatter(y="xG_p90", x="goals_p90", data_frame=season_wise_meta_df,
                 hover_name="season", text="Player", symbol="season",
                 opacity=.8)
fig.add_trace(go.Scatter(x=[season_wise_meta_df[["goals_p90", "xG_p90"]].min().min(),
                            season_wise_meta_df[["goals_p90", "xG_p90"]].max().max()],
                         y=[season_wise_meta_df[["goals_p90", "xG_p90"]].min().min(),
                            season_wise_meta_df[["goals_p90", "xG_p90"]].max().max()]))
fig.update_traces(textposition='top center', textfont_size=10, marker=dict(size=10))
fig.update_layout(showlegend=True, height=700, width=1000)

# The Final Comparison

In [ ]:
season_wise_meta_df

In [ ]:
radar_df = season_wise_meta_df.groupby(["Player"])[num_cols].sum().reset_index()

In [ ]:
radar_df

In [ ]:
radar_df.columns

In [ ]:
cols_for_radar = ['goals', 'shots', 'xG', 'xGDiff',
                  'assists', 'xA', 'key_passes',
                  'npg', 'npxG', 'goal_contrib', 'xgoal_contrib', 'shot_contrib']

## Radar Plot for Comparison Analysis

In [ ]:
# Initiate the plotly go figure
fig = go.Figure()
# Add Radar plots for different players:
fig.add_trace(go.Scatterpolar(
    r=radar_df.loc[(radar_df["Player"] == "Cristiano"), cols_for_radar].values.flatten(),
    theta=cols_for_radar,
    fill='toself',
    name='Cristiano'))
fig.add_trace(go.Scatterpolar(
    r=radar_df.loc[(radar_df["Player"] == "Messi"), cols_for_radar].values.flatten(),
    theta=cols_for_radar,
    fill='toself',
    name="Messi"))
# Additional properties for the plot:
fig.update_layout(
    title="Cristiano vs Messi",
polar=dict(
    radialaxis=dict(
    visible=True,
    )),
showlegend=True,
template="plotly_dark"
)
fig.show()

## Per90 Radar Plot

In [ ]:
per_90_cols

In [ ]:
cols_for_radar = ['goals_p90',
 'shots_p90',
 'xG_p90',
 'xGDiff_p90',
 'assists_p90',
 'xA_p90',
 'key_passes_p90',
 'npg_p90',
 'npxG_p90',
 'goal_contrib_p90',
 'xgoal_contrib_p90',
 'shot_contrib_p90']

In [ ]:
radar_df = season_wise_meta_df.groupby(["Player"])[cols_for_radar].sum().reset_index()

In [ ]:
# Initiate the plotly go figure
fig = go.Figure()
# Add Radar plots for different players:
fig.add_trace(go.Scatterpolar(
    r=radar_df.loc[(radar_df["Player"] == "Cristiano"), cols_for_radar].sum(),
    theta=cols_for_radar,
    fill='toself',
    name='Cristiano'))
fig.add_trace(go.Scatterpolar(
    r=radar_df.loc[(radar_df["Player"] == "Messi"), cols_for_radar].sum(),
    theta=cols_for_radar,
    fill='toself',
    name="Messi"))
# Additional properties for the plot:
fig.update_layout(
    title="Cristiano vs Messi",
polar=dict(
    radialaxis=dict(
    visible=True,
    )),
showlegend=True,
template="plotly_dark"
)
fig.show()

# Who Won?